# Inventory Agent with AWS services

## Overview
In this section, we will create an agent to check on the inventory of the products. The customer will be able to ask the agent for available clothes in the catalogue.

## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Native tools used   |aws_use                           |
|Agent Structure     |Single agent architecture                          |
|AWS services used   |Amazon DynamoDB

</div>


## Architecture

<div style="text-align:center">
    <img src="Inventory-agents.png" width="85%" />
</div>

## Key Features
* **Single agent architecture**: this example creates a single agent that interacts with built-in and custom tools
* **Connection with AWS services**: connects with Amazon DynamoDB for inventory
* **Bedrock Model as underlying LLM**: Used Amazon Nova from Amazon Bedrock as the underlying LLM model

## Setup and prerequisites

Let's now install the requirement packages and libraries for our Strands Agent

In [ ]:
# installing pre-requisites
!pip install --upgrade -q strands-agents
!pip install --upgrade -q strands-agents-tools
!pip install --upgrade -q boto3

In [ ]:
# import all libraries
import os
import boto3
from strands import Agent, tool
from strands.models import BedrockModel
from botocore.config import Config as BotocoreConfig
from strands_tools import use_aws, http_request

### Defining the agent

In [ ]:
model = BedrockModel(
    model_id="us.amazon.nova-premier-v1:0",
    region_name="us-west-2",
    temperature=0.3,
    top_p=0.8
)

In [ ]:
aws_agent = Agent(tools=[use_aws])

@tool
def get_product(product_id: str):
    """
    Use this tool when you need to get the details of a product to 
    see if that's available in stock or not.

    Args:
        product_id: The id of the product

    """
    print(f"Getting product from Dynamodb with the input {product_id}")

    get_item_result = aws_agent.tool.use_aws(
        service_name="dynamodb",
        operation_name="get_item",
        parameters={
            "TableName": "demo_mcp_product_inventory",
            "Key":{
                "product_id": {
                    "S": product_id
            }
            }
            },
        region="us-west-2",
        label="Get One Item"
    )
    return get_item_result


_system_prompt = """
You are an Agent that checks if a product is available in the inventory.
Return output in the below JSON format. Do not include any other text.
{
    "product_id" : PRODUCT_ID,
    "in_stock": IN_STOCK_VALUE
}

where IN_STOCK_VALUE = "yes" if quantity_available is > 0 and "no" otherwise.
"""

# Register the tool with the agent
inventory_agent = Agent(
    model=model,
    tools=[get_product],
    system_prompt=_system_prompt
)

## Invoking agent

Let's now test our agent

In [ ]:
product_agent_response = inventory_agent("What is the available quantity for PROD-048?")

#### Invoking agent with follow up question
Ok, let's now check more clothes and options

In [ ]:
quantity_agent_response = inventory_agent("How many PROD-028 are available?")

## Next Step

Congrats, you've completed lab 3. Next open `lab 4\search_orchestrator_agent` to continue.